In [31]:
# Data info
file_fold = r"D:\works\Data_analysis\projects\balance_beam\2023May\preprocessed_DLC_data\pose_estimation"
keypoints = ["start","end","nose","vertex","ear_R","spine_fore","spine_mid","spine_hind","tail_base","tail1","tail2","tail3","tail_tip","lateral_body","belly_fore","belly_mid","belly_hind","forelimb_R","forelimb_L","hindlimb_R","hindlimb_L"]
scorer = "DLC_resnet50_Balance_BeamMay29shuffle1_150000"

In [27]:
#convert preprocessed-csv to .h5

import os
import pandas as pd
from itertools import islice

def convertcsv2h5(csv_dir_path, scorer):
    for filename in os.listdir(csv_dir_path):
        if os.path.splitext(filename)[1] == ".csv":
            csv_file_path = os.path.join(csv_dir_path, filename)
            with open(csv_file_path) as datafile:
                head = list(islice(datafile, 0, 5))
            if "individuals" in head[1]:
                header = list(range(4))
            else:
                header = list(range(3))
            if head[-1].split(",")[0] == "labeled-data":
                index_col = [0, 1, 2]
            else:
                index_col = 0
            data = pd.read_csv(csv_file_path, index_col=index_col, header=header)
            data.columns = data.columns.set_levels([scorer], level="scorer")
            guarantee_multiindex_rows(data)
            data.to_hdf(csv_file_path.replace(".csv", ".h5"), key="df_with_missing", mode="w")
            data.to_csv(csv_file_path)


def guarantee_multiindex_rows(df):
    # Make paths platform-agnostic if they are not already
    if not isinstance(df.index, pd.MultiIndex):  # Backwards compatibility
        path = df.index[0]
        try:
            sep = "/" if "/" in path else "\\"
            splits = tuple(df.index.str.split(sep))
            df.index = pd.MultiIndex.from_tuples(splits)
        except TypeError:  #  Ignore numerical index of frame indices
            pass

    # Ensure folder names are strings
    try:
        df.index = df.index.set_levels(df.index.levels[1].astype(str), level=1)
    except AttributeError:
        pass

def robust_split_path(s):
    sep = "/" if "/" in s else "\\"
    return tuple(s.split(sep))

convertcsv2h5(file_fold,scorer)


In [33]:
import cebra
dlc_data = cebra.load_deeplabcut(r"D:\works\Data_analysis\projects\balance_beam\2023May\preprocessed_DLC_data\pose_estimation\0518_DoB0904A_baseline_cropped.h5", keypoints, pcutoff=0.6)

In [34]:
print(dlc_data)

[[ 190.92863464  138.32229614 1681.65600586 ...  125.32406616
     6.20021725   60.12770844]
 [ 191.03100586  138.31993866 1681.70245361 ...  125.22744751
    18.68275309   64.33098602]
 [ 191.09003448  138.41899109 1681.71075439 ...  125.12253952
    82.40930176  125.74793243]
 ...
 [ 194.23046112  137.30818176 1680.73986816 ...  136.14614868
  1649.51049805  129.21653748]
 [ 194.16217041  137.26894379 1681.70983887 ...  135.35223389
  1649.45458984  129.44108582]
 [ 194.08274841  137.23641968 1682.3671875  ...  135.0994873
  1657.95947266  129.77262878]]
